In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 40
#pd.set_option('display.max_rows', 200)

In [3]:
dataName = 'WCup2022_TeamData.csv'
DFTeamData = pd.read_csv(dataName)
DFTeamData

,Team,Group,Area,Ranking
0,Qatar,A,Host,50
1,Ecuador,A,SouthAmerica,44
2,Senegal,A,Africa,18
3,Netherlands,A,Europe,8
4,England,B,Europe,5
5,Iran,B,Asia,20
6,USA,B,Caribbean,16
7,Wales,B,Europe,19
8,Argentina,C,SouthAmerica,3
9,SaudiArabia,C,Asia,51


In [4]:
def Ranking(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    
    for team_index in Team_list:
        scored_sum = 0
        conceded_sum = 0
        winningPoint = 0
        for team_columns in Team_list:
            score = RoundRobin.at[team_index, team_columns]
            if type(score) is float:
                continue
            score_list = score.split(',')
            if len(score_list) != 2:
                continue
            
            scored, conceded = int(score_list[0]), int(score_list[1])
            scored_sum += scored
            conceded_sum += conceded
            if scored > conceded: winningPoint += 3
            elif scored == conceded: winningPoint += 1
                
        RoundRobin.at[team_index, 'WinningPoints'] = winningPoint
        RoundRobin.at[team_index, 'Score'] = scored_sum
        RoundRobin.at[team_index, 'Conceded'] = conceded_sum
        RoundRobin.at[team_index, 'GoalDiff'] = scored_sum - conceded_sum
        
    WinningPoints_rank = RoundRobin['WinningPoints'].rank(ascending=False, method='min')
    WinningPoints_list = WinningPoints_rank.tolist()
    RoundRobin['Ranking'] = WinningPoints_list
    Duplication_list1 = []
    for i in range(1,5):
        if WinningPoints_list.count(i) >= 2:
            Duplication_list1.append(i)
    for duplication1 in Duplication_list1:
        RoundRobin_duplication1 = RoundRobin.loc[RoundRobin['Ranking'] == duplication1]
        GoalDiff_rank = RoundRobin_duplication1['GoalDiff'].rank(ascending=False, method='min')
        GoalDiff_list = GoalDiff_rank.tolist()
        GoalDiff_index = GoalDiff_rank.index.tolist()
        for j in range(1,len(GoalDiff_list)+1):
            if GoalDiff_list.count(j) == 1:
                RoundRobin.at[GoalDiff_index[GoalDiff_list.index(j)], 'Ranking'] = j + duplication1-1
            elif GoalDiff_list.count(j) >= 2:
                idx = -1
                Duplication_list2 = []
                for cnt in range(GoalDiff_list.count(j)):
                    idx = GoalDiff_list.index(j, idx+1)
                    Duplication_list2.append(GoalDiff_index[idx])
                RoundRobin.loc[Duplication_list2, 'Ranking'] = j + duplication1-1
                RoundRobin_duplication2 = RoundRobin.loc[Duplication_list2]
                Score_rank = RoundRobin_duplication2['Score'].rank(ascending=False, method='min')
                Score_list = Score_rank.tolist()
                Score_index = Score_rank.index.tolist()
                for k in range(1,len(Score_list)+1):
                    if Score_list.count(k) == 1:
                        RoundRobin.at[Score_index[Score_list.index(k)], 'Ranking'] = k + j-1 + duplication1-1
    
    
    return RoundRobin

In [5]:
Score_list        = ['7,0'
                     , '7,1', '6,0'
                     , '7,2', '6,1', '5,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '2,7', '1,6', '0,5'
                     , '1,7', '0,6'
                     , '0,7']
Score_inverselist = ['0,7'
                     , '1,7', '0,6'
                     , '2,7', '1,6', '0,5'
                     , '3,7', '2.6', '1,5', '0,4'
                     , '4,7', '3,6', '2,5', '1,4', '0,3'
                     , '5,7', '4,6', '3,5', '2,4', '1,3', '0,2'
                     , '6,7', '5,6', '4,5', '3,4', '2,3', '1,2', '0,1'
                     , '7,7', '6,6', '5,5', '4,4', '3,3', '2,2', '1,1', '0,0'
                     , '7,6', '6,5', '5,4', '4,3', '3,2', '2,1', '1,0'
                     , '7,5', '6,4', '5,3', '4,2', '3,1', '2,0'
                     , '7,4', '6,3', '5,2', '4,1', '3,0'
                     , '7,3', '6,2', '5,1', '4,0'
                     , '7,2', '6,1', '5,0'
                     , '7,1', '6,0'
                     , '7,0']

In [6]:
def RankingProbability(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    #Score_list = ['5,0', '4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4', '0,5']
    #Score_list = ['1,0', '0,0', '0,1']
    # 11パターンで240秒(7分), 1秒で約670回
    # 予測：5試合分　13パターンで555秒x13=2時間
    # 予測：3試合分　55パターンで250秒x55=3.8時間, 64パターンで360秒x64=384分(6.4時間)

    X = np.array(range(len(Score_list)))
    
    Ranking_num = pd.DataFrame(0, columns=range(1,5), index=Team_list)
    Ranking_probability = pd.DataFrame(columns=range(1,5), index=Team_list)
    print("start : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
    continueSwitch1 = 0
    for i1 in X:
        if continueSwitch1 == 1: continue
        #print("0,1(" + str(i1) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
        RoundRobin1 = RoundRobin.copy()
        if RoundRobin1.iat[0,1] is float(np.nan):
            RoundRobin1.iat[0,1] = Score_list[i1]
            RoundRobin1.iat[1,0] = Score_inverselist[i1]
        else: continueSwitch1 = 1
        continueSwitch2 = 0
        for i2 in X:
            if continueSwitch2 == 1: continue
            #print("0,2(" + str(i2) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
            RoundRobin2 = RoundRobin1.copy()
            if RoundRobin2.iat[0,2] is float(np.nan):
                RoundRobin2.iat[0,2] = Score_list[i2]
                RoundRobin2.iat[2,0] = Score_inverselist[i2]
            else: continueSwitch2 = 1
            continueSwitch3 = 0
            for i3 in X:
                if continueSwitch3 == 1: continue
                #print("0,3(" + str(i3) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                RoundRobin3 = RoundRobin2.copy()
                if RoundRobin3.iat[0,3] is float(np.nan):
                    RoundRobin3.iat[0,3] = Score_list[i3]
                    RoundRobin3.iat[3,0] = Score_inverselist[i3]
                else: continueSwitch3 = 1
                continueSwitch4 = 0
                for i4 in X:
                    if continueSwitch4 == 1: continue
                    #print("1,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                    RoundRobin4 = RoundRobin3.copy()
                    if RoundRobin4.iat[1,2] is float(np.nan):
                        RoundRobin4.iat[1,2] = Score_list[i4]
                        RoundRobin4.iat[2,1] = Score_inverselist[i4]
                    else: continueSwitch4 = 1
                    continueSwitch5 = 0
                    for i5 in X:
                        if continueSwitch5 == 1: continue
                        #print("1,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                        RoundRobin5 = RoundRobin4.copy()
                        if RoundRobin5.iat[1,3] is float(np.nan):
                            RoundRobin5.iat[1,3] = Score_list[i5]
                            RoundRobin5.iat[3,1] = Score_inverselist[i5]
                        else: continueSwitch5 = 1
                        continueSwitch6 = 0
                        for i6 in X:
                            if continueSwitch6 == 1: continue
                            #print("2,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                            RoundRobin6 = RoundRobin5.copy()
                            if RoundRobin6.iat[2,3] is float(np.nan):
                                RoundRobin6.iat[2,3] = Score_list[i6]
                                RoundRobin6.iat[3,2] = Score_inverselist[i6]
                            else: continueSwitch6 = 1
                                
                            RoundRobin_rank = Ranking(RoundRobin6.copy())
                            
                            for j in range(1,5):
                                for k in range(0,4):
                                    if RoundRobin_rank.at[Team_list[k], 'Ranking'] == j:
                                        Ranking_num.at[Team_list[k], j] += 1
                            
    
    for k in range(0,4):
        patternNum = Ranking_num.loc[Team_list[k], :].sum()
        for j in range(1,5):
            Ranking_probability.at[Team_list[k], j] = str((Ranking_num.at[Team_list[k], j]/patternNum*100).round(1)) + '%'
        Ranking_probability.at[Team_list[k], 'Breakthrough'] = str(((Ranking_num.at[Team_list[k], 1]+Ranking_num.at[Team_list[k], 2])/patternNum*100).round(1)) + '%'
                            
    return Ranking_num, Ranking_probability

In [7]:
def OutputData(no):
    print(Score_list[no])
    Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '7,0' , '0,7'
    Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

    Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '1,1' , '1,1'
    Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '1,0', '0,1'

    Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = Score_inverselist[no] , Score_list[no]
    #Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
    Ranking(Group_E_RoundRobin)
    Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
    for i in range(1,5):
        Ranking_JapanPropability.at[Score_list[no], i] = Ranking_probability_E.at['Japan', i]
    Ranking_JapanPropability.at[Score_list[no], 'Breakthrough'] = Ranking_probability_E.at['Japan', 'Breakthrough']
    return Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E

In [8]:
Group_E_Team = DFTeamData.loc[DFTeamData['Group'] == 'E']
Group_E_Team_list = Group_E_Team['Team'].tolist()
Group_E_RoundRobin = pd.DataFrame(columns=Group_E_Team_list, index=Group_E_Team_list)
for team in Group_E_Team_list:
    Group_E_RoundRobin.at[team, team] = "-"

In [9]:
Ranking_JapanPropability = pd.DataFrame(columns=range(1,5))

In [10]:
no = 0
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,0
start : 2022.11.28 05:56:47


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,7",4.0,8.0,8.0,0.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,0","0,1","2,1",-,6.0,9.0,2.0,7.0,1.0


In [11]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,21.9%,78.1%,0.0%,21.9%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,35.9%,7.8%,56.2%,35.9%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [12]:
no = 1
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,1
start : 2022.11.28 05:56:47


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,7",4.0,9.0,8.0,1.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,1","0,1","2,1",-,6.0,9.0,3.0,6.0,1.0


In [13]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,32.8%,67.2%,0.0%,32.8%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,25.0%,18.8%,56.2%,25.0%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [14]:
no = 2
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,0
start : 2022.11.28 05:56:47


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,6",4.0,8.0,7.0,1.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,0","0,1","2,1",-,6.0,8.0,2.0,6.0,1.0


In [15]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,31.2%,68.8%,0.0%,31.2%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,26.6%,17.2%,56.2%,26.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [16]:
no = 3
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,2
start : 2022.11.28 05:56:47


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,7",4.0,10.0,8.0,2.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,2","0,1","2,1",-,6.0,9.0,4.0,5.0,1.0


In [17]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,40.6%,59.4%,0.0%,40.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,15.6%,28.1%,56.2%,15.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [18]:
no = 4
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,1
start : 2022.11.28 05:56:47


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,6",4.0,9.0,7.0,2.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,1","0,1","2,1",-,6.0,8.0,3.0,5.0,1.0


In [19]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,40.6%,59.4%,0.0%,40.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,17.2%,26.6%,56.2%,17.2%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [20]:
no = 5
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,0
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,5",4.0,8.0,6.0,2.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,0","0,1","2,1",-,6.0,7.0,2.0,5.0,1.0


In [21]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,39.1%,60.9%,0.0%,39.1%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,18.8%,25.0%,56.2%,18.8%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [22]:
no = 6
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,3
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,7",4.0,11.0,8.0,3.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,3","0,1","2,1",-,6.0,9.0,5.0,4.0,1.0


In [23]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,46.9%,53.1%,0.0%,46.9%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,9.4%,34.4%,56.2%,9.4%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [24]:
no = 7
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,2
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1",2.6,4.0,8.0,1.0,7.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,2","0,1","2,1",-,6.0,8.0,4.0,4.0,1.0


In [25]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [26]:
no = 8
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,1
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,5",4.0,9.0,6.0,3.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,1","0,1","2,1",-,6.0,7.0,3.0,4.0,1.0


In [27]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,46.9%,53.1%,0.0%,46.9%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,10.9%,32.8%,56.2%,10.9%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [28]:
no = 9
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,0
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,4",4.0,8.0,5.0,3.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,0","0,1","2,1",-,6.0,6.0,2.0,4.0,1.0


In [29]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,45.3%,54.7%,0.0%,45.3%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,12.5%,31.2%,56.2%,12.5%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [30]:
no = 10
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,4
start : 2022.11.28 05:56:48


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,7",4.0,12.0,8.0,4.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,4","0,1","2,1",-,6.0,9.0,6.0,3.0,1.0


In [31]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,51.6%,48.4%,0.0%,51.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,4.7%,39.1%,56.2%,4.7%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [32]:
no = 11
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,3
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,6",4.0,11.0,7.0,4.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,3","0,1","2,1",-,6.0,8.0,5.0,3.0,1.0


In [33]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,51.6%,48.4%,0.0%,51.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,4.7%,39.1%,56.2%,4.7%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [34]:
no = 12
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,2
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,5",4.0,10.0,6.0,4.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,2","0,1","2,1",-,6.0,7.0,4.0,3.0,1.0


In [35]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,51.6%,48.4%,0.0%,51.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,4.7%,39.1%,56.2%,4.7%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [36]:
no = 13
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,1
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,4",4.0,9.0,5.0,4.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,1","0,1","2,1",-,6.0,6.0,3.0,3.0,1.0


In [37]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,51.6%,48.4%,0.0%,51.6%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,6.2%,37.5%,56.2%,6.2%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [38]:
no = 14
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,0
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,3",4.0,8.0,4.0,4.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,0","0,1","2,1",-,6.0,5.0,2.0,3.0,1.0


In [39]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,50.0%,50.0%,0.0%,50.0%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,7.8%,35.9%,56.2%,7.8%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [40]:
no = 15
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,5
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,7",4.0,13.0,8.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,5","0,1","2,1",-,6.0,9.0,7.0,2.0,1.0


In [41]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [42]:
no = 16
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,4
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,6",4.0,12.0,7.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,4","0,1","2,1",-,6.0,8.0,6.0,2.0,1.0


In [43]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [44]:
no = 17
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,3
start : 2022.11.28 05:56:49


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,5",4.0,11.0,6.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,3","0,1","2,1",-,6.0,7.0,5.0,2.0,1.0


In [45]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [46]:
no = 18
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,2
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,4",4.0,10.0,5.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,2","0,1","2,1",-,6.0,6.0,4.0,2.0,1.0


In [47]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [48]:
no = 19
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,1
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,3",4.0,9.0,4.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,1","0,1","2,1",-,6.0,5.0,3.0,2.0,1.0


In [49]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,3.1%,40.6%,56.2%,3.1%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [50]:
no = 20
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,0
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,2",4.0,8.0,3.0,5.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,0","0,1","2,1",-,6.0,4.0,2.0,2.0,1.0


In [51]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,53.1%,46.9%,0.0%,53.1%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,4.7%,39.1%,56.2%,4.7%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [52]:
no = 21
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,6
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,7",4.0,14.0,8.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,6","0,1","2,1",-,6.0,9.0,8.0,1.0,1.0


In [53]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,0.0%,43.8%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [54]:
no = 22
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,5
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,6",4.0,13.0,7.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,5","0,1","2,1",-,6.0,8.0,7.0,1.0,1.0


In [55]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,100.0%,0.0%,0.0%,0.0%,100.0%


In [56]:
no = 23
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,4
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,5",4.0,12.0,6.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,4","0,1","2,1",-,6.0,7.0,6.0,1.0,1.0


In [57]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,98.4%,1.6%,0.0%,0.0%,100.0%


In [58]:
no = 24
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,3
start : 2022.11.28 05:56:50


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,4",4.0,11.0,5.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,3","0,1","2,1",-,6.0,6.0,5.0,1.0,1.0


In [59]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,98.4%,1.6%,0.0%,0.0%,100.0%


In [60]:
no = 25
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,2
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,3",4.0,10.0,4.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,2","0,1","2,1",-,6.0,5.0,4.0,1.0,1.0


In [61]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,0.0%,43.8%,56.2%,0.0%
Japan,98.4%,1.6%,0.0%,0.0%,100.0%


In [62]:
no = 26
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,1
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,2",4.0,9.0,3.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,1","0,1","2,1",-,6.0,4.0,3.0,1.0,1.0


In [63]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,56.2%,43.8%,0.0%,56.2%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,98.4%,1.6%,0.0%,0.0%,100.0%


In [64]:
no = 27
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,0
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,1",4.0,8.0,2.0,6.0,2.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,0","0,1","2,1",-,6.0,3.0,2.0,1.0,1.0


In [65]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,0.0%,54.7%,45.3%,0.0%,54.7%
CostaRica,1.6%,42.2%,12.5%,43.8%,43.8%
Germany,0.0%,1.6%,42.2%,56.2%,1.6%
Japan,98.4%,1.6%,0.0%,0.0%,100.0%


In [66]:
no = 28
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

7,7
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,7",5.0,15.0,8.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"7,7","0,1","2,1",-,4.0,9.0,9.0,0.0,2.0


In [67]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,34.4%,9.4%,56.2%,34.4%
Japan,0.0%,23.4%,76.6%,0.0%,23.4%


In [68]:
no = 29
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,6
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,6",5.0,14.0,7.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,6","0,1","2,1",-,4.0,8.0,8.0,0.0,2.0


In [69]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,35.9%,7.8%,56.2%,35.9%
Japan,0.0%,21.9%,78.1%,0.0%,21.9%


In [70]:
no = 30
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,5
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,5",5.0,13.0,6.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,5","0,1","2,1",-,4.0,7.0,7.0,0.0,2.0


In [71]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,37.5%,6.2%,56.2%,37.5%
Japan,0.0%,20.3%,79.7%,0.0%,20.3%


In [72]:
no = 31
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,4
start : 2022.11.28 05:56:51


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,4",5.0,12.0,5.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,4","0,1","2,1",-,4.0,6.0,6.0,0.0,2.0


In [73]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,39.1%,4.7%,56.2%,39.1%
Japan,0.0%,18.8%,81.2%,0.0%,18.8%


In [74]:
no = 32
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,3
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,3",5.0,11.0,4.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,3","0,1","2,1",-,4.0,5.0,5.0,0.0,2.0


In [75]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,40.6%,3.1%,56.2%,40.6%
Japan,0.0%,17.2%,82.8%,0.0%,17.2%


In [76]:
no = 33
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,2
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,2",5.0,10.0,3.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,2","0,1","2,1",-,4.0,4.0,4.0,0.0,2.0


In [77]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,42.2%,1.6%,56.2%,42.2%
Japan,0.0%,15.6%,84.4%,0.0%,15.6%


In [78]:
no = 34
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,1
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,1",5.0,9.0,2.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,1","0,1","2,1",-,4.0,3.0,3.0,0.0,2.0


In [79]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,14.1%,85.9%,0.0%,14.1%


In [80]:
no = 35
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,0
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","0,0",5.0,8.0,1.0,7.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,0","0,1","2,1",-,4.0,2.0,2.0,0.0,2.0


In [81]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,56.2%,43.8%,0.0%,0.0%,100.0%
CostaRica,43.8%,0.0%,12.5%,43.8%,43.8%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,12.5%,87.5%,0.0%,12.5%


In [82]:
no = 36
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

6,7
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,6",7.0,15.0,7.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"6,7","0,1","2,1",-,3.0,8.0,9.0,-1.0,2.0


In [83]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [84]:
no = 37
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,6
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,5",7.0,14.0,6.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,6","0,1","2,1",-,3.0,7.0,8.0,-1.0,2.0


In [85]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [86]:
no = 38
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,5
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,4",7.0,13.0,5.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,5","0,1","2,1",-,3.0,6.0,7.0,-1.0,2.0


In [87]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [88]:
no = 39
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,4
start : 2022.11.28 05:56:52


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,3",7.0,12.0,4.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,4","0,1","2,1",-,3.0,5.0,6.0,-1.0,2.0


In [89]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [90]:
no = 40
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,3
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,2",7.0,11.0,3.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,3","0,1","2,1",-,3.0,4.0,5.0,-1.0,2.0


In [91]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [92]:
no = 41
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,2
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,1",7.0,10.0,2.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,2","0,1","2,1",-,3.0,3.0,4.0,-1.0,2.0


In [93]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [94]:
no = 42
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,1
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","1,0",7.0,9.0,1.0,8.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,1","0,1","2,1",-,3.0,2.0,3.0,-1.0,2.0


In [95]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [96]:
no = 43
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

5,7
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,5",7.0,15.0,6.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"5,7","0,1","2,1",-,3.0,7.0,9.0,-2.0,2.0


In [97]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [98]:
no = 44
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,6
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,4",7.0,14.0,5.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,6","0,1","2,1",-,3.0,6.0,8.0,-2.0,2.0


In [99]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [100]:
no = 45
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,5
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,3",7.0,13.0,4.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,5","0,1","2,1",-,3.0,5.0,7.0,-2.0,2.0


In [101]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [102]:
no = 46
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,4
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,2",7.0,12.0,3.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,4","0,1","2,1",-,3.0,4.0,6.0,-2.0,2.0


In [103]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [104]:
no = 47
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,3
start : 2022.11.28 05:56:53


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,1",7.0,11.0,2.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,3","0,1","2,1",-,3.0,3.0,5.0,-2.0,2.0


In [105]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [106]:
no = 48
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,2
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","2,0",7.0,10.0,1.0,9.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,2","0,1","2,1",-,3.0,2.0,4.0,-2.0,2.0


In [107]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [108]:
no = 49
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

4,7
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,4",7.0,15.0,5.0,10.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"4,7","0,1","2,1",-,3.0,6.0,9.0,-3.0,2.0


In [109]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [110]:
no = 50
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,6
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,3",7.0,14.0,4.0,10.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,6","0,1","2,1",-,3.0,5.0,8.0,-3.0,2.0


In [111]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [112]:
no = 51
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,5
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,2",7.0,13.0,3.0,10.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,5","0,1","2,1",-,3.0,4.0,7.0,-3.0,2.0


In [113]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [114]:
no = 52
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,4
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,1",7.0,12.0,2.0,10.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,4","0,1","2,1",-,3.0,3.0,6.0,-3.0,2.0


In [115]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [116]:
no = 53
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,3
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","3,0",7.0,11.0,1.0,10.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,3","0,1","2,1",-,3.0,2.0,5.0,-3.0,2.0


In [117]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [118]:
no = 54
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

3,7
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,3",7.0,15.0,4.0,11.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"3,7","0,1","2,1",-,3.0,5.0,9.0,-4.0,2.0


In [119]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [120]:
no = 55
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2.6
start : 2022.11.28 05:56:54


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,2",7.0,14.0,3.0,11.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,2.6,"0,1","2,1",-,3.0,2.0,2.0,0.0,2.0


In [121]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [122]:
no = 56
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,5
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,1",7.0,13.0,2.0,11.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,5","0,1","2,1",-,3.0,3.0,7.0,-4.0,2.0


In [123]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [124]:
no = 57
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,4
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","4,0",7.0,12.0,1.0,11.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,4","0,1","2,1",-,3.0,2.0,6.0,-4.0,2.0


In [125]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [126]:
no = 58
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

2,7
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,2",7.0,15.0,3.0,12.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"2,7","0,1","2,1",-,3.0,4.0,9.0,-5.0,2.0


In [127]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [128]:
no = 59
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,6
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,1",7.0,14.0,2.0,12.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,6","0,1","2,1",-,3.0,3.0,8.0,-5.0,2.0


In [129]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [130]:
no = 60
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,5
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","5,0",7.0,13.0,1.0,12.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,5","0,1","2,1",-,3.0,2.0,7.0,-5.0,2.0


In [131]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [132]:
no = 61
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

1,7
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,1",7.0,15.0,2.0,13.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"1,7","0,1","2,1",-,3.0,3.0,9.0,-6.0,2.0


In [133]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [134]:
no = 62
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,6
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","6,0",7.0,14.0,1.0,13.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,3.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,6","0,1","2,1",-,3.0,2.0,8.0,-6.0,2.0


In [135]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,0.0%,43.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,100.0%,0.0%,0.0%


In [136]:
no = 63
Group_E_RoundRobin, Ranking_num_E, Ranking_probability_E = OutputData(no)
Group_E_RoundRobin

0,7
start : 2022.11.28 05:56:55


,Spain,CostaRica,Germany,Japan,WinningPoints,Score,Conceded,GoalDiff,Ranking
Spain,-,"7,0","1,1","7,0",7.0,15.0,1.0,14.0,1.0
CostaRica,"0,7",-,NaN,"1,0",3.0,1.0,7.0,-6.0,2.0
Germany,"1,1",NaN,-,"1,2",1.0,2.0,3.0,-1.0,4.0
Japan,"0,7","0,1","2,1",-,3.0,2.0,9.0,-7.0,3.0


In [137]:
Ranking_probability_E

,1,2,3,4,Breakthrough
Spain,100.0%,0.0%,0.0%,0.0%,100.0%
CostaRica,0.0%,56.2%,10.9%,32.8%,56.2%
Germany,0.0%,43.8%,0.0%,56.2%,43.8%
Japan,0.0%,0.0%,90.6%,9.4%,0.0%


In [138]:
Ranking_JapanPropability

,1,2,3,4,Breakthrough
"7,0",100.0%,0.0%,0.0%,0.0%,100.0%
"7,1",100.0%,0.0%,0.0%,0.0%,100.0%
"6,0",100.0%,0.0%,0.0%,0.0%,100.0%
"7,2",100.0%,0.0%,0.0%,0.0%,100.0%
"6,1",100.0%,0.0%,0.0%,0.0%,100.0%
"5,0",100.0%,0.0%,0.0%,0.0%,100.0%
"7,3",100.0%,0.0%,0.0%,0.0%,100.0%
"6,2",100.0%,0.0%,0.0%,0.0%,100.0%
"5,1",100.0%,0.0%,0.0%,0.0%,100.0%
"4,0",100.0%,0.0%,0.0%,0.0%,100.0%
